In [ ]:
from tbh import runner_tools as rt 
from tbh import plotting as pl

from importlib import reload
reload(rt);
reload(pl);

In [ ]:
params, priors = rt.get_parameters_and_priors()

In [ ]:
from tbh import model as tbm 

reload(tbm)

model_config = rt.DEFAULT_MODEL_CONFIG

model = tbm.get_tb_model(model_config)

In [ ]:
model.run(params)

In [ ]:
df = model.get_derived_outputs_df()

In [ ]:
df['notifications'].plot()